In [6]:
user = User.objects.first()
print(user.djstripe_customer.id)

cus_EsJLPjrnRQMUyB


In [ ]:
class client(object):
    
    def __init__(self, isSandbox):
        self.url = "https://api.bentoforbusiness.com"
        if isSandbox:
            self.url = "https://sandbox-api.bentoforbusiness.com/api"
        